In [ ]:
# question include new mode namley start_cv_run_building_id
# or introduces new parameter that can be use in the config file
# the latter is probably better
# pseudo code:
# start_cv_run_buidlingwise:
# if mode = group_cv_building
    # meter0 = train_df.meter == 0 returns logical vector
    # train_df = train_df meter0 rows
    # label = label aber nur meter 0 rows
# group kfold
# define the groups array by building_id
# and run the cross-validation

In [ ]:
def start_cv_run(train_df, label, params, splits, verbose_eval,
                 num_boost_round, early_stopping_rounds, output_filepath,
                 grouped_on_building = False):
    """
    Starts a Cross Validation Run with the parameters provided.
    Scores will be documented and models will be saved.
    :param train_df: DataFrame which contains the training data.
    :param label: A vector which contains the labels of the training data.
    :param params: Dictionary with the model parameters
    :param splits: Integer describing the number of folds / splitting fraction.
    :param verbose_eval: The interval where training information is printed
    to console.
    :param num_boost_round: Maximum number of rounds / estimators for the training.
    :param early_stopping_rounds: If no improvement of the validation score in
    n rounds occur, the training will be stopped.
    :param output_filepath: Directory that will contain the trained models.
    """
    output_filepath = output_filepath +  "_cv"
    cv_results = []
    with timer("Performing " + str(splits) + " fold cross-validation"):
        if grouped_on_building = True:
            # include groupkfold
            # kf = groupk
        else
            kf = KFold(n_splits=splits, shuffle=False, random_state=1337)
        for i, (train_index, test_index) in enumerate(kf.split(train_df, label)):
            with timer("~~~~ Fold %d of %d ~~~~" % (i + 1, splits)):
                x_train, x_valid = train_df.iloc[train_index], train_df.iloc[test_index]
                y_train, y_valid = label[train_index], label[test_index]

                train_lgb_df = lgb.Dataset(data=x_train, label=y_train)
                valid_lgb_df = lgb.Dataset(data=x_valid, label=y_valid)

                valid_sets = [train_lgb_df, valid_lgb_df]
                evals_result = dict()
                lgbm_model = lgb.train(params=params,
                                       train_set=train_lgb_df,
                                       num_boost_round=num_boost_round,
                                       valid_sets=valid_sets,
                                       valid_names=["train_loss", "eval"],
                                       verbose_eval=verbose_eval,
                                       evals_result=evals_result,
                                       early_stopping_rounds=early_stopping_rounds)
                save_model(output_filepath, lgbm_model)

                cv_results.append(evals_result)
        evaluate_cv_results(cv_results)

In [1]:
import numpy as np
from sklearn.model_selection import GroupKFold

In [6]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)

2

In [4]:
print(group_kfold)

GroupKFold(n_splits=4)


In [8]:
for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [0 1] TEST [2 3]
TRAIN: [2 3] TEST [0 1]
